In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
os.chdir('..')

# load daily close data for some ETFs
df = pd.read_csv('examples/ETF_prices.csv',index_col=0,parse_dates=True)
print('Loaded ', df.columns)

# resample to weekly frequency
weekly_prices = df.resample('W').ffill()

# compute compounded returns
weekly_returns = np.log(weekly_prices).diff().dropna()

# store the returns shape
T, N = weekly_returns.shape

# inputs/parameters
ANNUALIZATION_FACTOR = 52
HALF_LIFE = ANNUALIZATION_FACTOR * 10 # 10 years

Loaded  Index(['DBC', 'GLD', 'SPY', 'TLT'], dtype='object')


In [2]:
from pyvallocation import probabilities
from pyvallocation import moments

# generate uniform probabilities
p_uniform = probabilities.generate_uniform_probabilities(T)
print('Effective number of scenarios of probabilities with uniform dist:', probabilities.compute_effective_number_scenarios(p_uniform))

mu_uniform, cov_uniform = moments.estimate_sample_moments(weekly_returns,p_uniform)
mu_uniform_jorion = moments.shrink_mean_jorion(mu_uniform,cov_uniform,T)
cov_uniform_lw_cc = moments.shrink_covariance_ledoit_wolf(weekly_returns,cov_uniform,target='constant_correlation')

Effective number of scenarios of probabilities with uniform dist: 1006.0000000000003


In [3]:
from pyvallocation.views import FlexibleViewsProcessor, BlackLittermanProcessor

# Suppose we think S&P 500 will deliver a <2% annualized return in the future
fv = FlexibleViewsProcessor(
    prior_mean=mu_uniform_jorion,
    prior_cov=cov_uniform_lw_cc,
    # prior_returns=weekly_returns,
    # prior_probabilities=p_uniform,
    mean_views={("SPY"): ("<=", 0.02/ANNUALIZATION_FACTOR)},
    # corr_views={('SPY','GLD'):-0.2}
    # mean_views={'SPY':0.02/(ANNUALIZATION_FACTOR)}
    # mean_views=np.array([0.02,0.05,0.1,0.02])/ANNUALIZATION_FACTOR
)
(fv.get_posterior()[0]*ANNUALIZATION_FACTOR)

DBC    0.008065
GLD    0.024047
SPY    0.020000
TLT    0.010423
dtype: float64